# Simulated Annealing

A simulated annealing process to determine the optimal weight values of an artificial neuron.
Python 3.8+ is required.

```
function SIMULATED-ANNEALING(problem, schedule) returns a solution State
  current <- problem.INITIAL
  for t = 1 to inf do
      T <- schedule(t)
      if T = 0 then return current
      next <- a randomly selected successor of current
      delta_E <- VALUE(current) - VALUE(next)
      if delta_E > 0 then current <- next
      else current <- next only with probability e^(-delta_E/T)
```


## Development

Create the virtual environment, enter into it with `source`, and install the package locally:

```bash
$ python -m venv .venv
$ source .venv/Scripts/activate
(.venv)
$ python -m pip install -e ".[dev]"
```

See this reference for [installing Python packages](https://packaging.python.org/en/latest/tutorials/installing-packages/).

Now you can run the application from within the virtual environment:

```bash
python -m simulated_annealing
```

### Test Suite
Run tests:

```bash
(.venv)
$ python -m pytest -v tests/
```


### Documentation

Build documentation:

```bash
(.venv)
$ python -m sphinx -M html docs docs/_build
```


## Installation

Packaging and distributing a Python application is dependent on the target
operating system(s) and execution environment, which could be a Python virtual
environment, Linux container, or native application.

Install the application to a self-contained Python virtual environment:

```bash
(.venv)
$ python -m pip install <project source>
$ cp -r <project source>/etc .venv/
$ .venv/Scripts/simulated_annealing --help
```


## Execution

The installed application includes a wrapper script for command line execution.
The location of this scripts depends on how the application was installed.

---

### Configuration

The application uses `TOML` files for configuration. Configuration supports
runtime parameter substitution via a shell-like variable syntax, *i.e.*
`var = ${VALUE}`. CLI invocation will use the current environment for
parameter substitution, which makes it simple to pass host-specific values
to the application without needing to change the config file for every
installation.

```toml
    mailhost = $SENDMAIL_HOST
```

### Logging

The application uses standard `Python logging`. All loggins is to `STDERR`,
nd the logging level can be set via the config file or on the command line.


* [TOML](https://toml.io)
* [Python logging](https://docs.python.org/3/library/logging.html)

---



In [1]:
""" Implement the anneal command.

```
function SIMULATED-ANNEALING(problem, schedule) returns a solution State
  current <- problem.INITIAL
  for t = 1 to inf do
      T <- schedule(t)
      if T = 0 then return current
      next <- a randomly selected successor of current
      delta_E <- VALUE(current) - VALUE(next)
      if delta_E > 0 then current <- next
      else current <- next only with probability e^(-delta_E/T)
```

Stuart Russel, Peter Norvig. "Artificial Intelligence: A Modern Approach, 4th Edition" (2021)
"""
import numpy as np

from graphlib import TopologicalSorter
from typing import Callable, override

try:
    import matplotlib.pyplot as plt
    import networkx as nx
except ModuleNotFoundError:
    plt = None
    nx = None


class Neuron(object):
    DIM_W = 3

    def __init__(self, *weights:tuple[float]) -> None:
        """Initialize an instance of Neuron.

        :param weights: Tuple of weight values.
        :type weights: tuple[float]
        :raises TypeError: If weights is not of length .node.Neuron.DIM_W.
        """
        print("initializing %s", self.__class__.__name__)
        if weights is None or len(weights) != self.DIM_W:
            raise TypeError(f'weights : tuple[float] : Must be of length {self.DIM_W}.')
        self.weights = weights
        self.error = 2.

    def score(self, x, y) -> float:
        return 1 / (1 + np.exp(self.weights[0] * x + self.weights[1] * y + self.weights[2]))

    @override
    def __repr__(self) -> str:
        return f'Neuron#{hash(self)}:{self.weights}. err={self.error}'
 
    @override
    def __hash__(self) -> int:
        return hash(self.weights)

    @override
    def __eq__(self, other) -> bool:
        return self.weights == other.weights


class ProblemGraph(TopologicalSorter):

    def __init__(self, initial:Neuron) -> None:
        """Initialize an instance of ProblemGraph.

        :param initial: Neuron
        :type initial: Neuron
        :param evaluate: The evaluation function
        :type evaluate: Callable
        :raises TypeError: If initial is None
        """
        print("initializing %s", self.__class__.__name__)
        if initial is None:
            raise TypeError("initial : Neuron - Cannot be None")
        super().__init__({initial: {}})
        self.initial : Neuron = self._node2info[initial].node
        self.finished_graph = None

    def evaluate_node(self, node:Neuron) -> float:
        """Initialize a Neuron as a potential NAND gate.

        :param initial: Neuron
        :type initial: Neuron
        :return: Sum of errors for each combination
        :rtype: float
        """
        node.error = np.sum([
            np.abs(1 - node.score(0, 0)),
            np.abs(1 - node.score(0, 1)),
            np.abs(1 - node.score(1, 0)),
            np.abs(0 - node.score(1, 1))
        ])
        return node.error

    @property
    def graph(self) -> dict[Neuron, list[Neuron]]:
        if self.finished_graph is None:
            self.finished_graph = dict([(self._node2info[n].node, self._node2info[n].successors) for n in self._node2info])
        return self.finished_graph

    def __repr__(self) -> str:
        return f'ProblemGraph: {self.graph}'


def _anneal_step(problem:ProblemGraph, T:float, current:Neuron, successor:Neuron) -> Neuron:
    """ Execute one step of the simulated annealing function.
    
    :param problem: The problem definition.
    :type problem: ProblemGraph
    :param T: Current temperature.
    :type T: float
    :param current: The current node, searching for a greener pasture.
    :type current: Neuron
    :param successor: One of the neighboring nodes, enticing.
    :type successor: Neuron
    :return: Either the current or successor node
    :rtype: Neuron
    """
    delta_E = problem.evaluate_node(current) - problem.evaluate_node(successor)
    print(delta_E)
    if delta_E > 0:
        print("Taking successor as better option (exploitation)")
        problem.add(successor, current) # Trace the path
        return successor
    else:
        print(T)
        probability = np.exp(delta_E / T)
        if np.random.default_rng().uniform() < probability:
            print("Taking successor with probability %d%s (exploration)", probability*100, '%')
            problem.add(successor, current) # Trace the path
            return successor
    return current


def main(problem:ProblemGraph|None=None, schedule:Callable=lambda x : x / 1.002) -> ProblemGraph:
    """ Execute the simulated annealing algorithm.
    
    :param problem: The problem definition.
    :type problem: ProblemGraph
    :param schedule: Temperature function
    :type schedule: Callable
    :return: The problem with solution search graph.
    :rtype: ProblemGraph
    """
    print("executing anneal command")
    assert problem is None or type(problem) == ProblemGraph
    if problem is None:
        problem = ProblemGraph(Neuron(0, 0, 0))
    if schedule is None:
        schedule = lambda x : x / 1.2
    current = problem.initial
    print("Initial: %s", current)
    T = 1
    for t in range(10000000):
        T = schedule(T)
        if T < 0.00000001: break
        successor = Neuron(*current.weights + np.random.default_rng().uniform(low=-2.-current.error, high=2.+current.error, size=Neuron.DIM_W))
        current = _anneal_step(problem, T, current, successor)
    static_order = problem.static_order()
    #print(f"Static Order: {tuple(static_order)}")
    print("Winner: %s", current)
    print("Path Length: %s", len(problem.graph.keys()))
    #print("graph: %s", problem.graph)
    if plt is not None and nx is not None:
        G = nx.DiGraph()
        for v, e, in problem.graph.items():
            print("%s: %s", v,e)
    return problem


if __name__ == "__main__":
    main()



executing anneal command
initializing %s Neuron
initializing %s ProblemGraph
Initial: %s Neuron#3010437511937009226:(0, 0, 0). err=2.0
initializing %s Neuron
0.1756082704141293
Taking successor as better option (exploitation)
initializing %s Neuron
0.6990681070339291
Taking successor as better option (exploitation)
initializing %s Neuron
0.3260334362680797
Taking successor as better option (exploitation)
initializing %s Neuron
-0.6060038570879649
0.9920398405582133
initializing %s Neuron
-0.2271259548140876
0.9900597211159813
Taking successor with probability %d%s (exploration) 79.50054431762426 %
initializing %s Neuron
0.013246366852497138
Taking successor as better option (exploitation)
initializing %s Neuron
0.009717987262898076
Taking successor as better option (exploitation)
initializing %s Neuron
0.0032531593284446103
Taking successor as better option (exploitation)
initializing %s Neuron
-3.304638677015248e-05
0.9821786878790071
Taking successor with probability %d%s (exploratio